In [2]:
from prediction_reactions.service.preprocessing_data.prepearing_data import resources
%config Completer.use_jedi = False
import pandas as pd

from keras.models import Sequential
from keras import layers

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [3]:
data=pd.read_csv(resources+"data/Data_cleaning_with_content.csv")
data=data.dropna()
data.head()

,haha,sad,love,angry,wow,content,page_name,sumreactions,contentclean,len_text
0,10,12,1092,2,49,امطار خفيفة تهطل على العاصمة دمشق عدسة: Nour...,يوميات قذيفة هاون في دمشق,1165,امطار خفيفه تهطل العاصمه دمشق عدسه,34.0
1,195,1,315,0,5,. قاعدين بلا شغل وعم نقبض رواتب 🙂.. شو بدنا أح...,Lattakia Fire Brigade,516,قاعدين بلا شغل نقبض رواتب 🙂 احلى 🙃 اللاذقيه لل...,93.0
2,0,0,151,0,12,رادار البروق الآن | زخات مطر مصحوبة بالبروق بد...,Hawa Al Sham Weather هوى الشام,163,رادار البروق زخات مطر مصحوبه بالبروق بدات قليل...,742.0
3,0,1,5,3,4,🔊 نشرة أسعار الدولار 💯https://spprices.co...,الأسعار بالليرة السورية - SPPrices.com,13,نشره اسعار الدولار 💯,20.0
4,1,19,159,0,0,الشارع يتألم .. لكنه تائه في تحديد الخيارات .....,رفيق نصرالله - المركز الدولي للاعلام والدراسات,179,الشارع يتالم لكنه تائه تحديد الخيارات عندما يت...,193.0


In [4]:
data['percent_haha']=data['haha']/data['sumreactions']
data['percent_sad']=data['sad']/data['sumreactions']
data['percent_love']=data['love']/data['sumreactions']
data['percent_angry']=data['angry']/data['sumreactions']
data['percent_wow']=data['wow']/data['sumreactions']

In [5]:
my_data=data[['contentclean','percent_haha','percent_sad','percent_love','percent_angry','percent_wow']]

In [6]:
Y_train=my_data[['percent_haha','percent_sad','percent_love','percent_angry','percent_wow']].values

In [7]:
maxlen=100
tokenizer = Tokenizer(num_words=5000)

tokenizer.fit_on_texts(my_data['contentclean'].values)


X_train = tokenizer.texts_to_sequences(my_data['contentclean'].values)
    # X_test = tokenizer.texts_to_sequences(sentences_test)

        # Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1

        # Pad sequences with zeros
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)

In [8]:
def creat_model_cnn(vocab_size):
    
    model = Sequential()
    model.add(layers.Embedding(vocab_size, 50, input_length=100))
    model.add(layers.Conv1D(128, 5, activation='relu'))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(13,activation="softmax"))
    model.add(layers.Dense(5))
    model.compile(optimizer='adam',
                        loss='mse',
                    )
    return model
def creat_model_lstm(vocab_size):
    
    model = Sequential()
    model.add(layers.Embedding(vocab_size, 50, input_length=100))
    model.add(layers.Bidirectional(layers.LSTM(256)))
    model.add(layers.Flatten())
    model.add(layers.Dense(13,activation="softmax"))
    model.add(layers.Dense(5))
    model.compile(optimizer='adam',
                        loss='mse',
                         )
    return model

In [ ]:
model_cnn_regression=creat_model_cnn(vocab_size)
model_cnn_regression.fit(X_train,Y_train,batch_size=128,epochs=2)

In [ ]:
model_lstm_regression=creat_model_lstm(vocab_size)
model_lstm_regression.fit(X_train,Y_train,batch_size=128,epochs=2)

In [ ]:
model_cnn_regression.save(resources+"models/"+"model_cnn_regression.h5")
model_lstm_regression.save(resources+"models/"+"model_lstm_regression.h5")

In [ ]:
import pickle
with open(resources+"models/"+"tokenizer_model_cnn_lstm_regression.pickle","wb") as file:
    pickle.dump(tokenizer,file)